First, create the model. This must match the model used in the interactive training notebook.

In [ ]:
import cv2
import torch
import torchvision

CATEGORIES = ['apex']

device = torch.device('cuda')
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2 * len(CATEGORIES))
model = model.cuda().eval().half()

Next, load the saved model.  Enter the model path you used to save.

In [ ]:
model.load_state_dict(torch.load('vietdeptrai.pth'))
model.load_state_dict(torch.load('best_model_resnet18.pth'))

Convert and optimize the model using ``torch2trt`` for faster inference with TensorRT.  Please see the [torch2trt](https://github.com/NVIDIA-AI-IOT/torch2trt) readme for more details.

> This optimization process can take a couple minutes to complete. 

In [ ]:
from torch2trt import torch2trt

data = torch.zeros((1, 3, 224, 224)).cuda().half()

model_trt = torch2trt(model, [data], fp16_mode=True)

Save the optimized model using the cell below

In [ ]:
torch.save(model_trt.state_dict(), 'road_following_model_trt_1.pth')
torch.save(model_trt.state_dict(), 'best_model_resnet18_trt.pth')

Load the optimized model by executing the cell below

In [ ]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('road_following_model_trt_1.pth'))

Create the racecar class

Create the camera class.

Finally, execute the cell below to make the racecar move forward, steering the racecar based on the x value of the apex.

Here are some tips,

* If the car wobbles left and right,  lower the steering gain
* If the car misses turns,  raise the steering gain
* If the car tends right, make the steering bias more negative (in small increments like -0.05)
* If the car tends left, make the steering bias more postive (in small increments +0.05)

Kết hợp giữa tránh vật cản và theo dõi lane

In [ ]:
import torch
from torch2trt import TRTModule

# Load mô hình nhận diện vật cản
model_obstacle = TRTModule()
model_obstacle.load_state_dict(torch.load('best_model_resnet18_trt.pth'))

# Load mô hình chạy theo lane
model_steering = TRTModule()
model_steering.load_state_dict(torch.load('road_following_model_trt_1.pth'))



In [ ]:
import cv2
from torchvision.models import resnet18
from torch2trt import TRTModule
from jetracer.nvidia_racecar import NvidiaRacecar
from jetcam.csi_camera import CSICamera
from utils import preprocess
import torch.nn.functional as F
import numpy as np
import time
from IPython.display import display, clear_output

In [ ]:
from jetracer.nvidia_racecar import NvidiaRacecar
car = NvidiaRacecar()

# from jetcam.csi_camera import CSICamera
# camera = CSICamera(width=224, height=224, capture_fps=65)

In [ ]:
# Tham số điều khiển
STEERING_GAIN = 0.6
STEERING_BIAS = 0
THROTTLE = 0.2

# Biến trạng thái
prob_blocked = 0
threshold = 0.5

# Dừng xe ban đầu
car.throttle = 0.0
car.steering = 0.0
time.sleep(1)

while True:
    image = camera.read()
    image_proc = preprocess(image).half()

    # Nhận diện vật cản
    output_obstacle = model_obstacle(image_proc)
    output_lane = model_steering(image_proc).detach().cpu().numpy().flatten()
    y = F.softmax(output_obstacle, dim=1)
    prob_blocked = float(y.flatten()[0])  # Xác suất bị chặn

    if prob_blocked < threshold:
        # Nếu không có vật cản, chạy thẳng theo lane
        x = float(output_lane[0])
        car.steering = x * STEERING_GAIN + STEERING_BIAS
        car.throttle = THROTTLE  # Tiếp tục chạy
         # In thông tin góc lái
    else:
        # Nếu có vật cản, dừng lại
        car.steering = 0.0  # Làm thẳng lái
        car.throttle = 0.0  # Dừng xe